In [1]:
import torch
import time
from torch.utils.data import DataLoader
from torch import nn
from transformers import AdamW, WEIGHTS_NAME, get_linear_schedule_with_warmup
import csv
import numpy as np
import os
import logging
from fp16 import FP16_Module, FP16_Optimizer
from parallel import DataParallelModel, DataParallelCriterion
from collections import OrderedDict
from utils.utils import *
from utils.settings import args, TASK_DICT, init_logging, MODEL_CONFIG, MODEL_CLASS, SPECIAL_TOKENS, CONFIG_CLASS
from utils.settings import TOKENIZER, SPECIAL_TOKEN_IDS, FILL_VAL, SAVE_NAME, FINAL_SAVE_NAME, TOKENS_WEIGHT, CONFIG_NAME
from scheduler import AnnealingLR
from regularizers import REG_TYPES, REG_TYPE_KEYS, Weight_Regularized_AdamW, Weight_Regularized_SGD
from torch.nn import CrossEntropyLoss
from transformers.adapters import ConfigUnion, AdapterConfig, PrefixTuningConfig
logger = logging.getLogger(__name__)

/raid/cs21mtech11006/adapter-prompt/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_dir = "../../data"
sst = { "train":os.path.join(data_dir,"sst_to_squad-train-v2.0.json"),
           "eval":os.path.join(data_dir,"sst_to_squad-dev-v2.0.json"),
           "test":os.path.join(data_dir,"sst_to_squad-test-v2.0.json"),
      }

In [ ]:
import pandas as pd
sst_train = pd.read_json(sst["train"])

In [ ]:
sst_train["data"][0]